In [ ]:
!pip install sentence_transformers
!pip install language-tool-python
!pip install sentence_transformers
!pip install detoxify
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import re
from detoxify import Detoxify

     |████████████████████████████████| 79 kB 2.7 MB/s 
     |████████████████████████████████| 4.0 MB 10.8 MB/s 
     |████████████████████████████████| 1.2 MB 19.3 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 880 kB 44.3 MB/s 
     |████████████████████████████████| 596 kB 31.1 MB/s 
     |████████████████████████████████| 6.6 MB 34.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=d74a8064559cfc4e8354f88360224af9e034383f6abb8ee115ff09a32094f0cc
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=1ce3c842a5bb5a00488b4aa32677cfcbdf45c699ab1c1e302594d73b61d7dc26
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sentence-transformers sa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Check similatiries between two strings
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

def similar(a, b):
  first = model.encode(a)
  second = model.encode(b)
  return util.dot_score(first, second)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
#RESET THE DATAGRAME; BE CAREFULL!!!

#data = {"Joke" : ["Why was the math book sad? Because it had so many problems"], "Chiste": ["Van dos y se cae el del medio"]}
#post = pd.DataFrame(data)
#post.to_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Process Dataframes/Jokes.xlsx")

In [ ]:
#Read the data
post = pd.read_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Process Dataframes/Jokes.xlsx", index_col=0)

In [ ]:
#English Jokes

#Beutifull Soup
flag = 0
page = 0

#Extract 20 jokes
while flag < 20:

  #specify URL
  if page == 0:
    url = "http://www.laughfactory.com/jokes/clean-jokes/"
  else:
    url = "http://www.laughfactory.com/jokes/clean-jokes/{}".format(page)

  #get the HTML from the page and assign it to the html variable
  html = urlopen(url)

  #parse the HTML and assign it to the variable 'soup'
  soup = bs(html.read(), 'html.parser')

  #get the text of the joke and assign to the variable "chiste"
  #joke = soup.find("div", class_="jokes").text
  
  for jok in soup.find_all("div", class_ = "jokes"):
    
    joke = jok.find("div", class_ = "joke-text").text
    #print(joke)
    toxi = Detoxify('original').predict(str(joke))
    #If it contains any type of toxicity
    if any(v > 0.5 for v in iter(toxi.values())):
      continue

    #que sean chister cortos
    if len(joke) < 200:
      escape = False
      for chist in post.Joke:
          if similar(chist, joke)[0][0].item() > 0.9:
              escape = True
      if escape == False:
        if flag == 0:
          dat = {"Jokes": [joke.strip()]}
          jokes = pd.DataFrame(dat)
        else:   
          dat = {"Jokes": joke.strip()}
          jokes = jokes.append(dat, ignore_index=True)
        flag += 1

  #Avanzamos de pagina para la siguiente url
    
  page += 1

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt


  0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
#Chiste español 
#Beutifull Soup
flag = 0
#specify URL
url = "http://www.chistes.com/ChisteAlAzar.asp?n=3"

#Sacar 100 chistes
while flag < 20:
  #get the HTML from the page and assign it to the html variable
  html = urlopen(url)

  #parse the HTML and assign it to the variable 'soup'
  soup = bs(html.read(), 'html.parser')

  #get the text of the joke and assign to the variable "chiste"
  joke = soup.find("div", class_="chiste").text
  
  toxi = Detoxify('multilingual').predict(str(joke))
  #If it contains any type of toxicity
  if any(v > 0.7 for v in iter(toxi.values())):
    continue

  #que sean chister cortos
  if len(joke) < 200:
    escape = False
    for chist in post.Chiste:
        if similar(chist, joke)[0][0].item() > 0.9:
            escape = True
    if escape == False:
      if flag == 0:
        dat = {"Chiste": [joke]}
        chistes = pd.DataFrame(dat)
        flag += 1
      else:
        escape2 = False
        for chi in chistes.Chiste:
          if similar(joke,chi)[0][0].item() > 0.9:
            escape2 = True
        if escape2 == False:
          dat = {"Chiste": joke}
          chistes = chistes.append(dat, ignore_index=True)
          flag += 1

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.4-alpha/multilingual_debiased-0b549669.ckpt" to /root/.cache/torch/hub/checkpoints/multilingual_debiased-0b549669.ckpt


  0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

In [ ]:
n = 0
while (n < len(chistes)) & (n < len(jokes)):
  
  dat = {"Joke": (jokes["Jokes"][n]), "Chiste": (chistes["Chiste"][n])}
  post = post.append(dat, ignore_index=True)
  n += 1
post.to_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Process Dataframes/Jokes.xlsx")